# Thermography

## Libraries

In [479]:
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

## Data selection

In [480]:
class Data:
	def __init__(self, root):
		self.table = self.__generate_table(root)
		self.training, self.validation, self.test = None, None, None

	def __generate_table(self, root):
		files = []

		for category in os.listdir(root):
			for filename in os.listdir(root + category):
				files.append([root + category + "/" + filename, category])

		return pd.DataFrame(files, columns=["filename", "category"])

In [481]:
data = Data("./data/thermography/")

In [482]:
data.table.head(1)

,filename,category
0,./data/thermography/healthy/16-TFRON_V16_31-10...,healthy


## Preprocessing and transformation

In [483]:
data.table.category = data.table.category.factorize()[0].astype(str)

In [484]:
data.table.head(1)

,filename,category
0,./data/thermography/healthy/16-TFRON_V16_31-10...,0


In [485]:
data.training, data.test = train_test_split(data.table, test_size=0.15, shuffle=True)

In [486]:
print(f"Raw: {data.table.category.value_counts().values}")
print(f"Training: {data.training.category.value_counts().values}")
print(f"Test: {data.test.category.value_counts().values}")

Raw: [99 96]
Training: [83 82]
Test: [16 14]


In [487]:
train_datagen = ImageDataGenerator(
	rescale=1./255,
	validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [488]:
train_generator = train_datagen.flow_from_dataframe(
	dataframe=data.training,
    x_col="filename",
    y_col="category",
    target_size=(538, 215),
    batch_size=25,
    class_mode="categorical",
    shuffle=True,
    color_mode="rgb",
    subset="training"
)

validation_generator = train_datagen.flow_from_dataframe(
	dataframe=data.training,
    x_col="filename",
    y_col="category",
    target_size=(538, 215),
    batch_size=25,
    class_mode="categorical",
    shuffle=True,
    color_mode="rgb",
    subset="validation"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=data.test,
    x_col="filename",
    y_col="category",
    target_size=(538, 215),
    batch_size=1,
    shuffle=False,
    color_mode="rgb",
    class_mode="categorical"
)

Found 132 validated image filenames belonging to 2 classes.
Found 33 validated image filenames belonging to 2 classes.
Found 30 validated image filenames belonging to 2 classes.
